 Todo:
 1. remove columns with missing values with % > trashold using dropna with thresh
 2. impute missing values
 3. oversampling data
 4. run xgboost

In [18]:
############# READ DATA
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE


TRAIN_PATH = 'input/saftey_efficay_myopiaTrain.csv'
TEST_PATH = 'input/saftey_efficay_myopiaTest.csv'


train = pd.read_csv(TRAIN_PATH, low_memory=False)
train = train.dropna(axis=0, how='all') # remove missing rows
print("train.shape without missing rows =", train.shape)
# print(train.columns)
print("class count:\n",train.Class.value_counts())

train.shape without missing rows = (30451, 52)
class count:
 0.0    29949
1.0      502
Name: Class, dtype: int64


In [20]:
#### preprocess
y = train.Class
columns_to_drop = ['Class', 'Pre_L_Pupil_Day', 'T_L_Actual_AblDepth']
X = train.drop(columns=columns_to_drop)
dummies = pd.get_dummies(X)
X=(dummies-dummies.min())/(dummies.max()-dummies.min())
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X=imp_mean.fit_transform(X)

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.1, random_state=12)
sm = SMOTE(random_state=12, ratio = 1.0)
x_train_res, y_train_res = sm.fit_sample(x_train, y_train)  

          
# from sklearn.manifold import TSNE
# X_embedded = TSNE(n_components=2).fit_transform(X)

(53894, 72)

In [24]:
from sklearn.ensemble import RandomForestClassifier
foo = RandomForestClassifier(n_estimators=100, criterion='entropy')
foo.fit(x_train_res, y_train_res)
pred = foo.predict(x_val)
                                       

In [25]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(x_train_res, y_train_res)
y_pred = model.predict(x_val)


In [27]:
res = pred > 0
res.sum()

3

In [28]:
ress = y_pred > 0
ress.sum()

14